In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import seaborn as sns; sns.set()
import csv

from PIL import Image, ImageDraw
import os
import shutil
import glob

from mpl_toolkits.basemap import Basemap
from BetterMap import CreateMapBackground
import math
from statistics import mean 
import functools
import operator
import datetime
from pathlib import Path
from Utils import *

# Testing Audio
from IPython.lib.display import Audio

#there is currently a bug in panda's scatter_matrix that produces a warning that has no affect on what I'm doin :)
import warnings
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
C:\Users\aleja\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# TESTING ZONE

In [2]:
#load in the data
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False, delim_whitespace=True)
df.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)

df_center = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df_center.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)

In [3]:
#make a dataframe that denotes groups based on a time interval - a new interval every 'minutes_between' amount of minutes
minutes_between = 30

def add_instances_to_df(df, minutes_between):
    instance = 0
    df_time = pd.DataFrame()
    df_time['instance'] = 0
    for x in range(len(df['Month'].unique())):
        month = df['Month'].unique()[x]
        for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
            day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
            for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
                hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
                for w in range(0,60,minutes_between):
                    minute = w
                    temp = df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour) & (df['Min'] >= minute) & (df['Min'] <= minute + minutes_between)]
                    temp['instance'] = instance
                    df_time = df_time.append(temp, ignore_index=True)
                    instance+=1
    return df_time

df_time = add_instances_to_df(df, minutes_between)
df_time['distance'] = np.sqrt((df_time['Dist_East_West'] * df_time['Dist_East_West']) + 
                                (df_time['Dist_North_South'] * df_time['Dist_North_South']))
df_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230720 entries, 0 to 230719
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   instance          230720 non-null  int64  
 1   Year              230720 non-null  float64
 2   Month             230720 non-null  float64
 3   Day               230720 non-null  float64
 4   Hour              230720 non-null  float64
 5   Min               230720 non-null  float64
 6   Sec               230720 non-null  float64
 7   Lat               230720 non-null  float64
 8   Long              230720 non-null  float64
 9   Dist_East_West    230720 non-null  float64
 10  Dist_North_South  230720 non-null  float64
 11  distance          230720 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 21.1 MB


In [4]:
df_test = df_time.loc[df_time['instance'] == 1]
df_test

,instance,Year,Month,Day,Hour,Min,Sec,Lat,Long,Dist_East_West,Dist_North_South,distance
0,1,2017.0,8.0,27.0,17.0,56.0,9.7207,12.7452,-13.4960,271.57100,138.4600,304.831067
1,1,2017.0,8.0,27.0,17.0,56.0,19.4011,12.0782,-16.2206,-23.98660,64.2929,68.621673
2,1,2017.0,8.0,27.0,17.0,56.0,36.8910,9.6958,-8.5119,820.72900,-200.6180,844.892699
3,1,2017.0,8.0,27.0,17.0,56.0,42.7955,10.8790,-10.6200,587.47000,-69.0520,591.514311
4,1,2017.0,8.0,27.0,17.0,56.0,46.1723,9.9119,-12.6214,370.07400,-176.5890,410.046876
5,1,2017.0,8.0,27.0,17.0,56.0,47.9845,10.1117,-17.3002,-142.33000,-154.3720,209.972720
6,1,2017.0,8.0,27.0,17.0,56.0,47.9846,10.1071,-17.2740,-139.46400,-154.8830,208.420131
7,1,2017.0,8.0,27.0,17.0,56.0,48.2625,12.5963,-13.1908,304.84900,121.9030,328.318830
8,1,2017.0,8.0,27.0,17.0,57.0,6.6318,10.0730,-12.4941,383.82700,-158.6750,415.332303
9,1,2017.0,8.0,27.0,17.0,57.0,6.6318,10.1008,-12.5171,381.27600,-155.5840,411.798214


In [5]:
Mat = np.array(df_test.iloc[:,4:6])
print(Mat.shape)
Mat

(40, 2)


array([[17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 56.],
       [17., 57.],
       [17., 57.],
       [17., 57.],
       [17., 57.],
       [17., 57.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 58.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.],
       [17., 59.]])

In [ ]:
db = DBSCAN(eps=1.5, min_samples=3).fit(Mat)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool) 
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_ 
  
# Number of clusters in labels, ignoring noise if present. 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) 

# Black removed and is used for noise instead. 
unique_labels = set(labels) 

print(labels)
print(len(labels))
print(len(Mat))
print(n_clusters_)
#print(unique_labels)
print(Mat)

### Testing Cluster Centers

In [ ]:
cluster_lists_dict = {label: [Mat[i] for i in range(len(labels)) if labels[i] == label] for label in set(labels)}
cluster_center_dict = {label: [mean([x[0] for x in list_]), mean([x[1] for x in list_])] for label,list_ in cluster_lists_dict.items() }
cluster_center_dict

In [ ]:
colors = ['y', 'b', 'g', 'r', 'c', 'm', 'k', 'w'] 

In [ ]:
# MAP
img = np.asarray(mpimg.imread('MAP_ATLANTIC.PNG'))
plt.figure(figsize=(20,20))
plt.imshow(img,extent=[-100, 2.1734, 0, 41.3851])
# plt.imshow(image_to_plot,extent=[left_value, right_value, bottom_value, top_value])


# PLOT
for k, col in zip(unique_labels, colors): 
    if k == -1: 
        # Black used for noise. 
        col = 'k'
  
    class_member_mask = (labels == k) 
  
    xy = Mat[class_member_mask & core_samples_mask] 
    plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor= col, 
                                      markeredgecolor='k',  
                                      markersize=15) 
  
    xy = Mat[class_member_mask & ~core_samples_mask] 
    plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=col, 
                                      markeredgecolor='k', 
                                      markersize=15)

# CLUSTER CENTERS
for key, point in cluster_center_dict.items():
    if key != -1:
        plt.plot(point[1], point[0], 'o', markerfacecolor='w', 
                                          markeredgecolor='k', 
                                          markersize=15)
    
# EXECUTE
plt.title('number of clusters: %d' %n_clusters_) 
plt.show()

# RELEASE ZONE

In [ ]:
db_coords = []

In [ ]:
#make a dataframe that denotes groups based on a time interval - a new interval every 'minutes_between' amount of minutes

def add_instances_to_df(df, minutes_between):
    instance = 0
    df_time = pd.DataFrame()
    df_time['instance'] = 0
    for x in range(len(df['Month'].unique())):
        month = df['Month'].unique()[x]
        for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
            day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
            for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
                hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
                for w in range(0,60,minutes_between):
                    minute = w
                    temp = df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour) & (df['Min'] >= minute) & (df['Min'] <= minute + minutes_between)]
                    temp['instance'] = instance
                    df_time = df_time.append(temp, ignore_index=True)
                    instance+=1
    return df_time

In [ ]:
def DFtoGIF_DBSCAN( df_source, epsilon = 1.5, samples = 3, minutes = 30, di='./data/dbscan/final_gif/', buffer = 10):

    ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')
    year = 2017
    df = add_instances_to_df( df_source, minutes )
    colors = ['y', 'b', 'g', 'r', 'c', 'm', 'k']
    end = df['instance'].max()
    return_centroids = pd.DataFrame()
    
    img = np.asarray(mpimg.imread('MAP_ATLANTIC.PNG'))
    plt.figure(figsize=(20,20))
    plt.imshow(img,extent=[-100, 2.1734, 0, 41.3851])
    # plt.imshow(image_to_plot,extent=[left_value, right_value, bottom_value, top_value])

    if math.isnan(end):
        end = 0
        
    for current_segment in range(1, int(end)):
        
        df_temp = df[(df['instance'] == current_segment)]
        
        if(df_temp.shape[0] > 0):

            Mat = np.array(df_temp.iloc[:,4:6])
            db = DBSCAN( eps=epsilon, min_samples=samples ).fit(Mat)
            core_samples_mask = np.zeros_like(db.labels_, dtype=bool) 
            core_samples_mask[db.core_sample_indices_] = True
            labels = db.labels_ 

            # Number of clusters in labels, ignoring noise if present. 
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) 

            # Black removed and is used for noise instead. 
            unique_labels = set(labels) 
            
            # Cluster Centers
            cluster_lists_dict = {label: [Mat[i] for i in range(len(labels)) if labels[i] == label] for label in set(labels)}
            cluster_center_dict = {label: [mean([x[0] for x in list_]), mean([x[1] for x in list_])] for label,list_ in cluster_lists_dict.items() }
            
            # Store Cluster Centers in DF
            cluster_centers_aslist = [[current_segment, coordinate[0], coordinate[1], cluster_label] for cluster_label, coordinate in cluster_center_dict.items()]
            return_centroids = return_centroids.append(cluster_centers_aslist, ignore_index=True)            

            img = np.asarray(mpimg.imread('MAP_ATLANTIC.PNG'))
            fig = plt.figure(figsize=(20,20))
            plt.imshow(img,extent=[-100, 2.1734, 0, 41.3851])
            # plt.imshow(image_to_plot,extent=[left_value, right_value, bottom_value, top_value])

            for k, col in zip(unique_labels, colors): 
                if k == -1: 
                    # Black used for noise. 
                    col = 'k'

                class_member_mask = (labels == k) 

                xy = Mat[class_member_mask & core_samples_mask] 
                plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor= col, 
                                                  markeredgecolor='k',  
                                                  markersize=15) 

                xy = Mat[class_member_mask & ~core_samples_mask] 
                plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=col, 
                                                  markeredgecolor='k', 
                                                  markersize=15)
                
            # CLUSTER CENTERS
            for key, point in cluster_center_dict.items():
                if key != -1:
                    plt.plot(point[1], point[0], 'o', markerfacecolor='w', 
                                                      markeredgecolor='k', 
                                                      markersize=15)
            
            month, day, hour, minute = int(df_temp['Month'].iloc[0]), int(df_temp['Day'].iloc[0]), int(df_temp['Hour'].iloc[0]), int(df_temp['Min'].iloc[0])
            #Save and close the figure
            plt.savefig('{}{}_{}_{}_{}_{}'.format(di,year,month,day,hour,minute),bbox_inches='tight')
            plt.close(fig)
    return return_centroids

In [ ]:
%%time
framerate = 4410
play_time_seconds = 3
t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)

centroid_df = DFtoGIF_DBSCAN( df )

Audio(audio_data, rate=framerate, autoplay=True)

In [ ]:
centroid_df.shape

In [ ]:
MakeGif('./data/dbscan/final_gif', './data/dbscan/', 'dbscan_3', 100, 'png')

In [ ]:
ClearDir('./data/dbscan/final_gif')